In [2]:
# basic
import argparse
import os,datetime

# save things
import pandas as pd
from keras.callbacks import CSVLogger

# model_func
from tools.model_func import *

Using TensorFlow backend.


In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=""

# copied code

In [3]:
import tensorflow_hub as hub
from bert.tokenization import FullTokenizer
from keras import backend as K
from keras.layers import Layer

In [4]:
from tools.MyClock import MyClock
clk = MyClock()

In [5]:
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    bert_module =  hub.Module(bert_path)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    vocab_file, do_lower_case = sess.run(
        [
            tokenization_info["vocab_file"],
            tokenization_info["do_lower_case"],
        ]
    )

    return FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

# convert data for bert

In [6]:
import pickle

In [7]:
# tokenizer sequence to text keep <UNK\>
IN_DIR = 'data/sic_hierarchy'
x_train,y_trains,x_test,y_teast = get_input(in_dir=IN_DIR,mode='cat')

In [8]:
max_sequence_length = x_train.shape[1]

In [9]:
# get tokenizer
tok = pickle.load(open(os.path.join(IN_DIR,'tokenizer.pkl'),'rb'))
tok.index_word[0]=''
tok.index_word[1] = "w"*201 # trigger [UNK] in WordpieceTokenizer

In [10]:
sess = tf.Session()
tok_bert = create_tokenizer_from_hub_module()

W0806 13:11:17.914444 140404942452544 deprecation_wrapper.py:119] From /home/angela/xmtc/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [11]:
xx = np.concatenate([x_train,x_test])
x_gen = tok.sequences_to_texts_generator(xx)
N = xx.shape[0]
rows = []
clk.tic()
for i,text in enumerate(x_gen):
    dd = {}
    text = text.strip()
    tokens = tok_bert.tokenize(text)[:max_sequence_length-2] # truncate
    tokenized = ["[CLS]"] + tokens +["[SEP]"] # add [CLS],[SEP]
    sequence = tok_bert.convert_tokens_to_ids(tokenized) # convert to sequence
    pad_len = max([max_sequence_length-len(sequence),0]) # pad
    # add zero padding
    # input_mask = # mask zero paddings p
    dd['text'] = text
    dd['tokenized'] = tokenized
    dd['sequence'] = sequence + [0]*pad_len
    dd['mask'] = [1]*len(sequence) + [0]*pad_len
    rows.append(dd)
    if i%(N//100)==0:
        print("{} {:.2f}%".format(clk.toc(p=False),i/N*100),end='\r')

In [12]:
df = pd.read_pickle('data/sic_hierarchy.pkl')
ind = df[df['train/test']=='train'].index.to_list() + df[df['train/test']=='test'].index.to_list()
for i in range(len(rows)):
    rows[i]['id']=ind[i]
    if i < x_train.shape[0]:
        rows[i]['train/test']='train'
    else:
        rows[i]['train/test']='test'

In [13]:
bert_df = pd.DataFrame.from_dict(rows)
bert_df.to_pickle('data/sic_hierarchy/bert_x.pkl')

In [14]:
bert_df.head()

,id,mask,sequence,text,tokenized,train/test
0,453844,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 6160, 2000, 17350, 13695, 2658, 1998, 10...",welcome to a1 lifts professional and reliable ...,"[[CLS], welcome, to, a1, lifts, professional, ...",train
1,914843,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2256, 5379, 2326, 11341, 2846, 1997, 484...",our friendly service surprising range of fresh...,"[[CLS], our, friendly, service, surprising, ra...",train
2,326959,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 11210, 2017, 2215, 2000, 2131, 2000, 199...",wherever you want to get to in life lloyds ban...,"[[CLS], wherever, you, want, to, get, to, in, ...",train
3,445157,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2024, 2017, 2559, 2005, 2152, 3737, 2655...",are you looking for high quality dentistry in ...,"[[CLS], are, you, looking, for, high, quality,...",train
4,530932,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1037, 2088, 1997, 3325, 1516, 2057, 2024...",a world of experience – we are experts in crui...,"[[CLS], a, world, of, experience, –, we, are, ...",train


# make small data for gdrive

In [4]:
df = pd.read_pickle('data/sic_hierarchy/bert_x.pkl')
df.head()

,id,mask,sequence,text,tokenized,train/test
0,453844,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 6160, 2000, 17350, 13695, 2658, 1998, 10...",welcome to a1 lifts professional and reliable ...,"[[CLS], welcome, to, a1, lifts, professional, ...",train
1,914843,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2256, 5379, 2326, 11341, 2846, 1997, 484...",our friendly service surprising range of fresh...,"[[CLS], our, friendly, service, surprising, ra...",train
2,326959,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 11210, 2017, 2215, 2000, 2131, 2000, 199...",wherever you want to get to in life lloyds ban...,"[[CLS], wherever, you, want, to, get, to, in, ...",train
3,445157,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2024, 2017, 2559, 2005, 2152, 3737, 2655...",are you looking for high quality dentistry in ...,"[[CLS], are, you, looking, for, high, quality,...",train
4,530932,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1037, 2088, 1997, 3325, 1516, 2057, 2024...",a world of experience – we are experts in crui...,"[[CLS], a, world, of, experience, –, we, are, ...",train


In [5]:
df = pd.concat([df[df['train/test']=='train'],df[df['train/test']=='test']],ignore_index=True)

In [7]:
_,y_trains,_,y_tests = get_input('data/sic_hierarchy','cat',get_output=[0,1,0,1])
y_seq_1 = [np.argmax(y,axis=1) for y in y_trains]
y_seq_2 = [np.argmax(y,axis=1) for y in y_tests]
y_seq = [np.concatenate([y_seq_1[i],y_seq_2[i]]) for i in range(len(y_seq_1))]
df['cat'] = np.array(y_seq).T.tolist()
df.head()

In [9]:
_,y_trains,_,y_tests = get_input('data/sic_hierarchy','hierarchy',get_output=[0,1,0,1])
y_seq_1 = [np.argmax(y,axis=1) for y in y_trains]
y_seq_2 = [np.argmax(y,axis=1) for y in y_tests]
y_seq = [np.concatenate([y_seq_1[i],y_seq_2[i]]) for i in range(len(y_seq_1))]
df['hierarchy'] = np.array(y_seq).T.tolist()
df.head()

,id,mask,sequence,text,tokenized,train/test,cat,hierarchy
0,453844,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 6160, 2000, 17350, 13695, 2658, 1998, 10...",welcome to a1 lifts professional and reliable ...,"[[CLS], welcome, to, a1, lifts, professional, ...",train,"[0, 23, 179, 203]",[203]
1,914843,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2256, 5379, 2326, 11341, 2846, 1997, 484...",our friendly service surprising range of fresh...,"[[CLS], our, friendly, service, surprising, ra...",train,"[4, 33, 248, 278]",[278]
2,326959,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 11210, 2017, 2215, 2000, 2131, 2000, 199...",wherever you want to get to in life lloyds ban...,"[[CLS], wherever, you, want, to, get, to, in, ...",train,"[8, 47, 331, 383]",[383]
3,445157,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 2024, 2017, 2559, 2005, 2152, 3737, 2655...",are you looking for high quality dentistry in ...,"[[CLS], are, you, looking, for, high, quality,...",train,"[14, 66, 413, 497]",[497]
4,530932,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1037, 2088, 1997, 3325, 1516, 2057, 2024...",a world of experience – we are experts in crui...,"[[CLS], a, world, of, experience, –, we, are, ...",train,"[11, 60, 379, 455]",[455]


In [10]:
df = df.drop(columns=['mask','tokenized','text'])

In [12]:
df.to_pickle('data/sic_hierarchy/bert_gdrive.pkl')

# temp

In [ ]:
import time
class MyClock:
    def __init__(self,fmt='mm:ss'):
        self.fmt = fmt
    def tic(self):
        self.start = time.time()
    def toc(self,p = True):
        t = time.time()
        elapsed = t-self.start
        if p:
            print('TIME ELAPSED: {}'.format(self._get_print(elapsed)))
        else:
            return self._get_print(elapsed)
    
    def _get_print(self,tt):
        if self.fmt == 'mm:ss':
            return '{:02d}:{:02d}'.format(int(tt//60),int(tt%60))
